Name : Kamaleshwar Viyanwar

Roll no: C3-43

Practical-7

In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
text = "Give papa a cup of proper coffee in a copper coffee mug."
tokens = word_tokenize(text.lower())

In [ ]:
# Generate bigrams
bigrams = list(ngrams(tokens, 2))
print("Bigrams:", bigrams)

Bigrams: [('give', 'papa'), ('papa', 'a'), ('a', 'cup'), ('cup', 'of'), ('of', 'proper'), ('proper', 'coffee'), ('coffee', 'in'), ('in', 'a'), ('a', 'copper'), ('copper', 'coffee'), ('coffee', 'mug'), ('mug', '.')]


In [ ]:
# Generate trigrams
trigrams = list(ngrams(tokens, 3))
print("Trigrams:", trigrams)


Trigrams: [('give', 'papa', 'a'), ('papa', 'a', 'cup'), ('a', 'cup', 'of'), ('cup', 'of', 'proper'), ('of', 'proper', 'coffee'), ('proper', 'coffee', 'in'), ('coffee', 'in', 'a'), ('in', 'a', 'copper'), ('a', 'copper', 'coffee'), ('copper', 'coffee', 'mug'), ('coffee', 'mug', '.')]


In [ ]:
def preprocess_text(text):
    # Convert to lowercase and remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

In [ ]:
def generate_ngrams(words, n):
    return list(ngrams(words, n))

In [ ]:
def get_ngram_frequencies(ngrams_list):
    return collections.Counter(ngrams_list)

In [ ]:
def laplace_smoothing(ngram_freqs, vocab_size, total_ngrams, ngram):
    ngram_count = ngram_freqs[ngram] if ngram in ngram_freqs else 0
    return (ngram_count + 1) / (total_ngrams + vocab_size)

In [ ]:
def good_turing_discounting(ngram_freqs, total_ngrams, ngram, ngram_count):
    count_of_n_count = sum(1 for count in ngram_freqs.values() if count == ngram_count)
    if ngram_count == 0:
        return count_of_n_count / total_ngrams
    else:
        return (ngram_count + 1) * (count_of_n_count + 1) / (total_ngrams * count_of_n_count)


In [ ]:
class BackOffModel:
    def __init__(self, ngram_freqs, vocab_size, total_ngrams, smoothing_func):
        self.ngram_freqs = ngram_freqs
        self.vocab_size = vocab_size
        self.total_ngrams = total_ngrams
        self.smoothing_func = smoothing_func
        self.cache = {}

    def get_probability(self, ngram):
        if ngram in self.cache:
            return self.cache[ngram]

        ngram_count = self.ngram_freqs[ngram] if ngram in self.ngram_freqs else 0
        probability = self.smoothing_func(self.ngram_freqs, self.vocab_size, self.total_ngrams, ngram)
        self.cache[ngram] = probability
        return probability

In [ ]:
words=preprocess_text(text)

In [ ]:
#unigrams
n = 1
ngrams_list = generate_ngrams(words, n)
ngrams_list

[('give',),
 ('papa',),
 ('cup',),
 ('proper',),
 ('coffee',),
 ('copper',),
 ('coffee',),
 ('mug',)]

In [ ]:
n = 2
bigrams_list = generate_ngrams(words, n)
bigrams_list

[('give', 'papa'),
 ('papa', 'cup'),
 ('cup', 'proper'),
 ('proper', 'coffee'),
 ('coffee', 'copper'),
 ('copper', 'coffee'),
 ('coffee', 'mug')]

In [ ]:
n = 3
trigrams_list = generate_ngrams(words, n)
trigrams_list

[('give', 'papa', 'cup'),
 ('papa', 'cup', 'proper'),
 ('cup', 'proper', 'coffee'),
 ('proper', 'coffee', 'copper'),
 ('coffee', 'copper', 'coffee'),
 ('copper', 'coffee', 'mug')]

In [ ]:
bigram_freqs = get_ngram_frequencies(bigrams_list)
total_bigrams = sum(bigram_freqs.values())
vocab_size = len(set(words))
total_bigrams


7

In [ ]:
bigram_freqs

Counter({('give', 'papa'): 1,
         ('papa', 'cup'): 1,
         ('cup', 'proper'): 1,
         ('proper', 'coffee'): 1,
         ('coffee', 'copper'): 1,
         ('copper', 'coffee'): 1,
         ('coffee', 'mug'): 1})

In [ ]:
vocab_size

7

In [ ]:
for bigram in bigrams_list:
    prob = laplace_smoothing(bigram_freqs, vocab_size, total_bigrams, bigram)
    print(f"Laplace smoothed probability for {bigram}: {prob:.4f}")

Laplace smoothed probability for ('give', 'papa'): 0.1429
Laplace smoothed probability for ('papa', 'cup'): 0.1429
Laplace smoothed probability for ('cup', 'proper'): 0.1429
Laplace smoothed probability for ('proper', 'coffee'): 0.1429
Laplace smoothed probability for ('coffee', 'copper'): 0.1429
Laplace smoothed probability for ('copper', 'coffee'): 0.1429
Laplace smoothed probability for ('coffee', 'mug'): 0.1429


In [ ]:
# Apply Good-Turing discounting
for bigram in bigrams_list:
    prob = good_turing_discounting(bigram_freqs, total_bigrams, bigram, 1)
    print(f"Good-Turing discounting for {bigram}: {prob:.4f}")



Good-Turing discounting for ('give', 'papa'): 0.3265
Good-Turing discounting for ('papa', 'cup'): 0.3265
Good-Turing discounting for ('cup', 'proper'): 0.3265
Good-Turing discounting for ('proper', 'coffee'): 0.3265
Good-Turing discounting for ('coffee', 'copper'): 0.3265
Good-Turing discounting for ('copper', 'coffee'): 0.3265
Good-Turing discounting for ('coffee', 'mug'): 0.3265


In [ ]:
# Initialize the Back-Off Model
backoff_model = BackOffModel(bigram_freqs, vocab_size, total_bigrams, laplace_smoothing)

# Apply Back-Off Model
for bigram in bigrams_list:
    prob = backoff_model.get_probability(bigram)
    print(f"Back-Off Model for {bigram}: {prob:.4f}")


Back-Off Model for ('give', 'papa'): 0.1429
Back-Off Model for ('papa', 'cup'): 0.1429
Back-Off Model for ('cup', 'proper'): 0.1429
Back-Off Model for ('proper', 'coffee'): 0.1429
Back-Off Model for ('coffee', 'copper'): 0.1429
Back-Off Model for ('copper', 'coffee'): 0.1429
Back-Off Model for ('coffee', 'mug'): 0.1429
